# Cómo quedaría ahora el código con las métricas: Métricas de Minkowski y Dynamic Time Warping (DTW)

Introducir Minkowski y DTW cambia el juego, especialmente para el modelo kNN. Estas métricas definen cómo el algoritmo mide la "similitud" entre dos granjas o situaciones de cultivo.

Aquí tienes el código actualizado y la explicación de qué implica cada una:

1. **Código Completo con Minkowski y DTW**

Para usar DTW, necesitamos la librería tslearn (especializada en series temporales), ya que Scikit-learn no la trae por defecto.

In [6]:
# 1. Desinstalamos para evitar conflictos
!pip uninstall -y tslearn sklearn scikit-learn

# 2. Instalamos una versión de scikit-learn compatible (1.2.2 o 1.3.2 suelen funcionar)
!pip install scikit-learn==1.3.2

# 3. Instalamos tslearn y numba (que es su motor de cálculo)
!pip install tslearn numba

Found existing installation: tslearn 0.7.0
Uninstalling tslearn-0.7.0:
  Successfully uninstalled tslearn-0.7.0
Found existing installation: scikit-learn 1.8.0
Uninstalling scikit-learn-1.8.0:
  Successfully uninstalled scikit-learn-1.8.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 3.8 MB/s eta 0:00:0000:0100:01
  Using cached tslearn-0.7.0-py3-none-any.whl.metadata (16 kB)
  Using cached scikit_learn-1.8.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (11 kB)
Using cached tslearn-0.7.0-py3-none-any.whl (372 kB)
Using cached scikit_learn-1.8.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (9.1 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2


In [1]:
# Instalación necesaria: pip install tslearn
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tslearn.neighbors import KNeighborsTimeSeriesClassifier # Para DTW

# 1. Carga y preprocesamiento base
df = pd.read_csv('smart_agriculture_bangladesh.csv')
df = df.drop_duplicates().dropna().reset_index(drop=True)
df.drop(['device_id', 'timestamp'], axis=1, inplace=True)

# Encoding y Escalado (Obligatorio para estas métricas de distancia)
le = LabelEncoder()
for col in ['location', 'crop_type', 'season', 'crop_health', 'pest_risk']:
    df[col] = le.fit_transform(df[col])

X = df.drop(['irrigation_needed'], axis=1)
y = df['irrigation_needed']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Partición 80/20
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# --- MÉTRICA 1: MINKOWSKI ---
# La distancia de Minkowski es una generalización. 
# Si p=1 es Manhattan, si p=2 es Euclídea. Usaremos p=3 para el ejemplo.
knn_minkowski = KNeighborsClassifier(n_neighbors=7, metric='minkowski', p=3)
cv_minkowski = cross_val_score(knn_minkowski, X_train, y_train, cv=5)

# --- MÉTRICA 2: DTW (Dynamic Time Warping) ---
# DTW es costosa computacionalmente. Se usa si los datos se ven como secuencias.
# Nota: Requiere que X_train tenga forma de serie (n_samples, n_timestamps, n_features)
X_train_dtw = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
knn_dtw = KNeighborsTimeSeriesClassifier(n_neighbors=7, metric="dtw")
# (Ojo: ejecutar DTW en datasets grandes puede tardar mucho)

print(f"Precisión media Minkowski (p=3): {cv_minkowski.mean():.4f}")

Precisión media Minkowski (p=3): 0.9635


In [ ]:
from tslearn.utils import to_time_series_dataset

# 1. Convertimos tus datos de agricultura a formato de serie temporal (n_muestras, n_timestamps, n_características)
X_train_dtw = to_time_series_dataset(X_train)
X_test_dtw = to_time_series_dataset(X_test)

# 2. Instanciamos el modelo DTW
# 'sakoe_chiba' con un radio pequeño hace que el cálculo sea MUCHO más rápido
knn_dtw = KNeighborsTimeSeriesClassifier(n_neighbors=7, 
                                          metric="dtw", 
                                          metric_params={"global_constraint": "sakoe_chiba", "itakura_max_slope": 2},
                                          n_jobs=-1) # Usa todos tus procesadores

# 3. Entrenamos (Ten paciencia, DTW es lento)
knn_dtw.fit(X_train_dtw, y_train)

# 4. Evaluamos
score = knn_dtw.score(X_test_dtw, y_test)
print(f"Precisión con métrica DTW: {score:.4f}")

---

**2\. Explicación de las Métricas**

#### **Métrica de Minkowski**

Es la "madre" de las distancias. Su fórmula es:

$$D(x, y) \= \\left( \\sum\_{i=1}^{n} |x\_i \- y\_i|^p \\right)^{1/p}$$

* **Si $p=1$:** Es la distancia **Manhattan** (movimiento en cuadrícula). Útil si tienes muchas variables y quieres que las diferencias grandes en una sola variable no dominen tanto.  
* **Si $p=2$:** Es la **Euclídea** (línea recta).  
* **Si $p \\geq 3$:** Se vuelve más sensible a las diferencias individuales más grandes entre las características del suelo.

#### **Dynamic Time Warping (DTW)**

A diferencia de las anteriores, DTW no compara punto a punto de forma rígida.

* **En Agricultura:** Imagina que dos sensores detectan una subida de temperatura. Uno la detecta a las 12:00 y otro a las 12:15 debido a la sombra de una nube.  
* La distancia Euclídea diría que son "diferentes" porque no coinciden en el tiempo exacto.  
* **DTW** "estira" las curvas para encontrar que el patrón es el mismo, aunque esté desplazado. Es ideal si analizas la evolución del cultivo a lo largo de varios días.

---

**3\. Discusión: ¿Valen la pena en este dataset?**

1. **Minkowski ($p=3$):** Puede ayudar si crees que hay una variable (como la humedad crítica) que debe penalizar mucho el error cuando se desvía, pero rara vez mejora drásticamente a la Euclídea ($p=2$).  
2. **DTW:** En este dataset específico (que parece ser de registros puntuales y no de series temporales largas por cada planta), **DTW es excesivo**. Hará que tu código sea 100 veces más lento sin aportar mucha más precisión, ya que cada fila es independiente de la anterior.

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder

# 1. Carga y preprocesamiento
df = pd.read_csv('smart_agriculture_bangladesh.csv')
df = df.drop_duplicates().dropna().reset_index(drop=True)
df.drop(['device_id', 'timestamp'], axis=1, inplace=True)

# Encoding
le = LabelEncoder()
for col in ['location', 'crop_type', 'season', 'crop_health', 'pest_risk']:
    df[col] = le.fit_transform(df[col])

X = df.drop(['irrigation_needed'], axis=1)
y = df['irrigation_needed']

# Escalado (CRÍTICO para Minkowski)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2. Configuración de Modelos con Minkowski (p=1 y p=2)
seed = 42
kfold = KFold(n_splits=5, shuffle=True, random_state=seed)

# p=1: Manhattan
knn_manhattan = KNeighborsClassifier(n_neighbors=7, metric='minkowski', p=1)
# p=2: Euclídea
knn_euclidean = KNeighborsClassifier(n_neighbors=7, metric='minkowski', p=2)

# 3. Evaluación
cv_manhattan = cross_val_score(knn_manhattan, X_scaled, y, cv=kfold)
cv_euclidean = cross_val_score(knn_euclidean, X_scaled, y, cv=kfold)

print(f"Precisión media Manhattan (p=1): {cv_manhattan.mean():.4f}")
print(f"Precisión media Euclídea (p=2):  {cv_euclidean.mean():.4f}")

Precisión media Manhattan (p=1): 0.9664
Precisión media Euclídea (p=2):  0.9646
